In [1]:
class BlogCrawler:
    def __init__(self, url):
        headers = {"User-Agent": "Mozilla/5.0"}
        req = urllib.request.Request(url, headers=headers)
        self._soup = BeautifulSoup(urllib.request.urlopen(req).read(), "html.parser")

    @property
    def info(self):
        return {
            "title": self.title,
            "description": self.description,
            "image": self.image,
        }

    @property
    def title(self):
        return self._soup.find("meta", property="og:title")["content"]

    @property
    def description(self):
        return self._soup.find("meta", property="og:description")["content"]

    @property
    def image(self):
        return self._soup.find("meta", property="og:image")["content"]

In [2]:
from utils import remove_escape
from crawler.base import BlogCrawler


class MediumCrawler(BlogCrawler):
    def __init__(self, url):
        BlogCrawler.__init__(self, url)

    @property
    def content(self):
        return remove_escape((self._soup.select_one("article")).get_text())

ModuleNotFoundError: No module named 'utils'

In [29]:
import re
import json

def url_tuples(mail):
    with open(mail, "r", encoding="utf-8") as file:
        data = file.read()  
    # 정규표현식 패턴
    pattern = r"(.*)\((https?://[^\s)]+)\)\n\n(.*)"

    # 매칭
    matches = re.findall(pattern, data)

    # 결과를 튜플로 저장
    result = [(match[0].strip() if match[0] else None, 
            match[1].strip() if match[1] else None, 
            match[2].strip() if match[2] else None) for match in matches]

    return result
    
urls = url_tuples('mail.txt')

In [33]:
urls

[('·Member',
  'https://medium.com/@pandapear557?source=email-5e68d4753404-1733850148089-digest.reader-------------------------0d7bcddd_6297_43be_859d_bfce301a76fb',
  "Today's highlights"),
 ('inJavaScript in Plain English',
  'https://medium.com/javascript-in-plain-english?source=email-5e68d4753404-1733850148089-digest.reader-4b3a1ed4f11c-ecd698da3dbd----0-102------------------0d7bcddd_6297_43be_859d_bfce301a76fb-1',
  'How to Generate Insights from PDF Files with Apryse and GPT'),
 ('inCyberArk Engineering',
  'https://medium.com/cyberark-engineering?source=email-5e68d4753404-1733850148089-digest.reader-9a668e42362e-7bda34c7f4e4----1-98------------------0d7bcddd_6297_43be_859d_bfce301a76fb-1',
  'Building Production-Ready AI Agents with LangGraph: A Real-Life Use Case'),
 ('inWord Garden',
  'https://medium.com/word-garden?source=email-5e68d4753404-1733850148089-digest.reader-956a0ce9b0f1-d6656699ef4d----2-98------------------0d7bcddd_6297_43be_859d_bfce301a76fb-1',
  '7 Writing Sid

In [34]:
def slugify(article_title): #이런 함수를 만들어도 괜찮은걸까..
    # 모든 \uXXXX 형태의 Unicode escape sequence 제거
    article_title = article_title.encode("unicode_escape").decode("ascii")  
    article_title = re.sub(r'\\u[0-9a-fA-F]{4}', ' ', article_title)
    article_title = re.sub(r'[^\w\s]', '', article_title)

    # 공백을 '-'로 변환, 압축
    article_title = article_title.strip().lower().replace(' ', '-')
    article_title = re.sub(r'-+', '-', article_title)

    return article_title

In [30]:
import re

def convert_text(input_text):
    # Convert text to lowercase
    input_text = input_text.lower()
    
    # Replace spaces with hyphens
    input_text = input_text.replace(" ", "-")
    
    # Remove non-alphanumeric characters except hyphens
    input_text = re.sub(r'[^a-z0-9-]', '', input_text)
    
    return input_text


In [31]:
def make_url(urls, idx):
    # 튜플 요소 분리
    author_name, base_url, article_title = urls[idx]

    # 제목을 슬러그 형식으로 변환 (소문자, 공백은 -, 특수문자 제거)
    slug = convert_text(article_title)

    # 최종 URL 생성
    final_url = f"{base_url.split('?')[0]}/{slug}-{base_url.split('?')[1].split('.reader-')[1].split('-')[1]}?{base_url.split('?')[1]}"

    return author_name, final_url

In [59]:
for i in range(19):  print(urls[i][2])
make_url(urls,-5)

Today's highlights
How to Generate Insights from PDF Files with Apryse and GPT
Building Production-Ready AI Agents with LangGraph: A Real-Life Use Case
7 Writing Side Hustles That Pay My Rent As A Complete Beginner
I Gained 15,000 Followers With These 3 Dead Simple Steps
AI train your face for cheap in 5 minutes (never make images of yourself again)
'It's a Wonderful Life' and Wartime America
How I Make Money From Crypto - Without Trading
The Secret Code Google Uses To Monitor Everything You Do Online😲
Get Paid $10,000 Monthly with the Amazon Associates Program
Bluesky and enshittification
Four Levels of RAG - Research from Microsoft
Apple Finally Got a Permanent Replacement for Adobe Photoshop and Illustrator
My Simple Content Formula That Generates $9,500 Monthly From Writing
Still using the 'You are an expert… ' AI prompt Part 2
The Medium Newsletter (https://medium.com/@dailynewsletter?source=email-5e68d4753404-1733850148089-digest.reader-15f753907972-ae53b2a22bf2----0-1-----------

('inGenerative AI',
 'https://medium.com/generative-ai/still-using-the-you-are-an-expert--ai-prompt-part-2-30839d6163ed?source=email-5e68d4753404-1733850148089-digest.reader-440100e76000-30839d6163ed----13-98------------------0d7bcddd_6297_43be_859d_bfce301a76fb-1')

In [52]:
temp = []
for i in range(len(urls)-1):
        temp.append(make_url(urls,i))
temp

[('·Member',
  'https://medium.com/@pandapear557/todays-highlights-?source=email-5e68d4753404-1733850148089-digest.reader-------------------------0d7bcddd_6297_43be_859d_bfce301a76fb'),
 ('inJavaScript in Plain English',
  'https://medium.com/javascript-in-plain-english/how-to-generate-insights-from-pdf-files-with-apryse-and-gpt-ecd698da3dbd?source=email-5e68d4753404-1733850148089-digest.reader-4b3a1ed4f11c-ecd698da3dbd----0-102------------------0d7bcddd_6297_43be_859d_bfce301a76fb-1'),
 ('inCyberArk Engineering',
  'https://medium.com/cyberark-engineering/building-production-ready-ai-agents-with-langgraph-a-real-life-use-case-7bda34c7f4e4?source=email-5e68d4753404-1733850148089-digest.reader-9a668e42362e-7bda34c7f4e4----1-98------------------0d7bcddd_6297_43be_859d_bfce301a76fb-1'),
 ('inWord Garden',
  'https://medium.com/word-garden/7-writing-side-hustles-that-pay-my-rent-as-a-complete-beginner-d6656699ef4d?source=email-5e68d4753404-1733850148089-digest.reader-956a0ce9b0f1-d6656699e

In [69]:
from bs4 import BeautifulSoup
import urllib

test = "https://medium.com/word-garden/7-writing-side-hustles-that-pay-my-rent-as-a-complete-beginner-d6656699ef4d"

headers = {"User-Agent": "Mozilla/5.0"}
req = urllib.request.Request(test, headers=headers)
soup = BeautifulSoup(urllib.request.urlopen(req).read(), "html.parser")

titles = soup.select_one("h1").get_text()
paragraphs = soup.select(".pw-post-body-paragraph")
txt = "\n".join([paragraph.get_text() for paragraph in paragraphs])

print(txt)

I never thought I’d say this, but writing actually pays my rent now.
Trust me, I’m no Shakespeare — just someone who discovered these seven writing jobs that don’t need fancy degrees or years of experience.
Remember that time I bought those bright yellow running shoes that looked amazing online but felt like concrete blocks?
Well, now I get paid to write honest product reviews so others don’t make my mistakes.
What you’ll do:
Where to start:


In [183]:
headers = {"User-Agent": "Mozilla/5.0"}
req = urllib.request.Request(temp, headers=headers)
soup = BeautifulSoup(urllib.request.urlopen(req).read(), "html.parser")

titles = soup.select_one("h1").get_text()
paragraphs = soup.select(".pw-post-body-paragraph")
txt = "\n".join([paragraph.get_text() for paragraph in paragraphs])
print(titles, '\n\n' ,txt)

Text to Video AI: How to Create Videos for Free — A Complete Guide 

 Text to Video generation has been the trend these days and a lot of popular tools are available for this use-case such as Vadoo, Invideo, Pictory etc. But most of these tools are paid and the budget can quickly go out of reach when you need multiple revisions to your output video
So in this article we shall discuss how we can create videos from text for free using an open-source solution https://github.com/SamurAIGPT/Text-To-Video-AI . If you prefer just watching a demo of using the project without getting into the coding aspects here is a tutorial video for the same
Now let’s get technical. Below are the entire workflow for generating a complete video from text
If you prefer to go for a premium voice you can try Elevenlabs api instead of edgetts. If you are on a system with lower hardware specifications, you can skip using whisper locally and use whisper api instead. With the entire workflow being clear, let’s under

In [18]:
!pip install flask

In [20]:
from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route('/process', methods=['POST'])
def process_json():
    # JSON 파일 확인
    if not request.is_json:
        return jsonify({"error": "Invalid JSON format"}), 400

    # JSON 데이터 읽기
    data = request.get_json()

    # JSON 데이터 처리 (예: 텍스트 추출)
    processed_text = f"Received keys: {', '.join(data.keys())}"
    
    return jsonify({"response": processed_text}), 200

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=8080)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8080
 * Running on http://172.30.1.84:8080
Press CTRL+C to quit
172.30.1.84 - - [16/Dec/2024 17:18:36] "GET / HTTP/1.1" 404 -
172.30.1.84 - - [16/Dec/2024 17:18:36] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [16/Dec/2024 17:18:40] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [16/Dec/2024 17:18:40] "GET /favicon.ico HTTP/1.1" 404 -
